# 584. Find Customer Referee

In [1]:
# Pandas Schema

import pandas as pd

data = [[1, 'Will', None], [2, 'Jane', None], [3, 'Alex', 2], [4, 'Bill', None], [5, 'Zack', 1], [6, 'Mark', 2]]
customer = pd.DataFrame(data, columns=['id', 'name', 'referee_id']).astype(
    {'id': 'Int64', 'name': 'object', 'referee_id': 'Int64'})

In [2]:
customer

,id,name,referee_id
0,1,Will,<NA>
1,2,Jane,<NA>
2,3,Alex,2
3,4,Bill,<NA>
4,5,Zack,1
5,6,Mark,2


In [3]:
#To pyspark schema

from pyspark.sql.types import *
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

# Spark has issues with null values when using Pandas' Int64 data type, hence creating a Spark schema
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("referee_id", IntegerType(), True)
])

In [4]:
spark = SparkSession.builder.getOrCreate()

customer_df = spark.createDataFrame(data, schema=schema)
customer_df.show(truncate=False)

bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
23/11/05 10:39:38 WARN Utils: Your hostname, svchost resolves to a loopback address: 127.0.1.1; using 172.18.28.34 instead (on interface eth0)
23/11/05 10:39:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/05 10:39:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+---+----+----------+
|id |name|referee_id|
+---+----+----------+
|1  |Will|null      |
|2  |Jane|null      |
|3  |Alex|2         |
|4  |Bill|null      |
|5  |Zack|1         |
|6  |Mark|2         |
+---+----+----------+


In [5]:
# in Spark Dataframe
customer_df.filter((F.col('referee_id').isNull()) | (F.col('referee_id') != 2)).select('name').show()

+----+
|name|
+----+
|Will|
|Jane|
|Bill|
|Zack|
+----+


In [6]:
# in Spark SQL
customer_df.createOrReplaceTempView('customer')
spark.sql('''
select name from customer where referee_id is null or referee_id!=2;
''').show()

+----+
|name|
+----+
|Will|
|Jane|
|Bill|
|Zack|
+----+


In [7]:
spark.stop()